# Why is Python so slow\*?

\* relative to other languages, for interpreted, dynamically typed code, etc, etc.

## Tracing CPython code execution

In [ ]:
def foo(a, b):
    return a + b

Let's disassemble it with the `dis` module:

In [ ]:
from dis import dis
dis(foo)

### What's `BINARY_ADD`?

We crack open CPython's source code and take a look inside `Python/ceval.c`:

```c
/* Python/ceval.c */
TARGET(BINARY_ADD) {
    PyObject *right = POP();
    PyObject *left = TOP();
    PyObject *sum;
    /* NOTE(haypo): Please don't try to micro-optimize int+int on
       CPython using bytecode, it is simply worthless.
       See http://bugs.python.org/issue21955 and
       http://bugs.python.org/issue10044 for the discussion. In short,
       no patch shown any impact on a realistic benchmark, only a minor
       speedup on microbenchmarks. */
    if (PyUnicode_CheckExact(left) &&
            PyUnicode_CheckExact(right)) {
        sum = unicode_concatenate(left, right, f, next_instr);
        /* unicode_concatenate consumed the ref to left */
    }
    else {
        sum = PyNumber_Add(left, right); // <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
        Py_DECREF(left);
    }
    Py_DECREF(right);
    SET_TOP(sum);
    if (sum == NULL)
        goto error;
    DISPATCH();
}
```

### What's `PyNumber_Add(left, right)`?

```c
/* Objects/abstract.c */
PyObject *
PyNumber_Add(PyObject *v, PyObject *w)
{
    PyObject *result = binary_op1(v, w, NB_SLOT(nb_add)); // <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
    if (result == Py_NotImplemented) {
        PySequenceMethods *m = v->ob_type->tp_as_sequence;
        Py_DECREF(result);
        if (m && m->sq_concat) {
            return (*m->sq_concat)(v, w);
        }
        result = binop_type_error(v, w, "+");
    }
    return result;
}
```

### What's `binary_op1()`?

```c
static PyObject *
binary_op1(PyObject *v, PyObject *w, const int op_slot)
{
    PyObject *x;
    binaryfunc slotv = NULL;
    binaryfunc slotw = NULL;

    if (v->ob_type->tp_as_number != NULL)
        slotv = NB_BINOP(v->ob_type->tp_as_number, op_slot);
    if (w->ob_type != v->ob_type &&
        w->ob_type->tp_as_number != NULL) {
        slotw = NB_BINOP(w->ob_type->tp_as_number, op_slot); // <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
        if (slotw == slotv)
            slotw = NULL;
    }
    if (slotv) {
        if (slotw && PyType_IsSubtype(w->ob_type, v->ob_type)) {
            x = slotw(v, w);
            if (x != Py_NotImplemented)
                return x;
            Py_DECREF(x); /* can't do it */
            slotw = NULL;
        }
        x = slotv(v, w);
        if (x != Py_NotImplemented)
            return x;
        Py_DECREF(x); /* can't do it */
    }
    if (slotw) {
        x = slotw(v, w);
        if (x != Py_NotImplemented)
            return x;
        Py_DECREF(x); /* can't do it */
    }
    Py_RETURN_NOTIMPLEMENTED;
}
```

### What's `NB_BINOP()`?

```c
#define NB_BINOP(nb_methods, slot) \
        (*(binaryfunc*)(& ((char*)nb_methods)[slot]))
```

### Cut to the chase: where's the addition function for two ints (longs)?

```c
/* Objects/longobject.c */
static PyObject *
long_add(PyLongObject *a, PyLongObject *b)
{
    PyLongObject *z;

    CHECK_BINOP(a, b);

    if (Py_ABS(Py_SIZE(a)) <= 1 && Py_ABS(Py_SIZE(b)) <= 1) {
        return PyLong_FromLong(MEDIUM_VALUE(a) + MEDIUM_VALUE(b)); // <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
    }
    if (Py_SIZE(a) < 0) {
        if (Py_SIZE(b) < 0) {
            z = x_add(a, b);
            if (z != NULL) {
                /* x_add received at least one multiple-digit int,
                   and thus z must be a multiple-digit int.
                   That also means z is not an element of
                   small_ints, so negating it in-place is safe. */
                assert(Py_REFCNT(z) == 1);
                Py_SIZE(z) = -(Py_SIZE(z));
            }
        }
        else
            z = x_sub(b, a);
    }
    else {
        if (Py_SIZE(b) < 0)
            z = x_sub(a, b);
        else
            z = x_add(a, b);
    }
    return (PyObject *)z;
}
```

### What's `MEDIUM_VALUE()`?

```c
/* Objects/longobject.c */
#define MEDIUM_VALUE(x) (assert(-1 <= Py_SIZE(x) && Py_SIZE(x) <= 1),  \
     Py_SIZE(x) < 0 ? -(sdigit)(x)->ob_digit[0] :  \
         (Py_SIZE(x) == 0 ? (sdigit)0 :  \
          (sdigit)(x)->ob_digit[0]))
```

## Why so much code for such a simple operation?

* C-level is interpreting bytecodes (`BINARY_ADD` in this case).
* Polymorphism -- code can handle `foo('a', 'b')` or any types that support `+`.
* Works for user-defined types, too, with an `__add__` or `__radd__` magic method.
* Error checking everywhere...
* For adding ints, does overflow checking and conversions, etc.

All these features mean a lot of code at the C level.

## What is the performance?

In [ ]:
%timeit foo(1, 2)

In [ ]:
%timeit foo('a', 'b')

## Summary

### CPython slowdown 1: interpreted bytecode execution

* Fetching CPython bytecode ops, managing the stack machine, all ops in `ceval.c`.
* Extensive error checking and handling.

### CPython slowdown 2: dynamic type resolution

* Type introspection, dynamic dispatch on every operation / method call, supporting generic operations.
* And extensive error checking and handling, up-and-down the call stack.

How does Cython speed it up? [02-cython-comparison.ipynb](02-cython-comparison.ipynb)